In [ ]:
from keras.applications import VGG16
# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows, img_cols = 224, 224 

# Re-loads the Vgg16 model without the top or FC layers
model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

In [ ]:
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

In [ ]:
#freezing all layers except the top layers

In [ ]:
from keras.applications import VGG16
# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows, img_cols = 224, 224 

# Re-loads the Vgg16 model without the top or FC layers
model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows,img_cols, 3))
#making the default layers from trainable to false
for layers in model.layers:
    layer.trainable = False
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)  

In [ ]:
def addTopModel(bottom_model, num_classes,D=256):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    #top_model = GlobalAveragePooling2D()(top_model)
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(D,activation='relu')(top_model)
    #top_model = Dense(1024,activation='relu')(top_model)
    #top_model = Dense(512,activation='relu')(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model


num_classes = 2

FC_Head = addTopModel(model, num_classes)

modelnew = Model(inputs = model.input, outputs = FC_Head)

print(modelnew.summary())

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'D:\\New folder\\veer\\Train'
validation_data_dir = 'D:\\New folder\\veer\\test'


#  using data augmentaiton 

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# set our batch size (typically on most mid tier systems we'll use 16-32)
train_batchsize = 4
val_batchsize = 4
img_rows=224
img_cols=224
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize ,
        class_mode='categorical') #binary    
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize ,
        class_mode= 'categorical',
        shuffle=False)

In [ ]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

                     
checkpoint = ModelCheckpoint("veerpal_vgg.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

#earlystop = EarlyStopping(monitor = 'val_loss', 
                          #min_delta = 0, 
                          #patience = 3,
                          #verbose = 1,
                          #restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [checkpoint]

# We use a very small learning rate 
modelnew.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 200
nb_validation_samples = 48

# We only train 5 EPOCHS 
epochs = 5
batch_size = 16

history = modelnew.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

modelnew.save("veerpal_vgg.h5")

In [ ]:
from keras.models import load_model
classifier = load_model("task4_vgg.h5")

In [ ]:
train_generator.class_indices

In [ ]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile,join

face_dict = {"[0]": "Rishi",
                      "[1]": "Veer"}

face_dict_n = {"Rishi":"Rishi",
                        "Veer":"Veer"}

def draw_test(name,predt,im):
    face=face_dict[str(predt)]
    BLACK = [0,0,0]
    expand = cv2.copyMakeBorder(im,40,40,40,40,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expand,face,(10,30),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
    cv2.imshow(name,expand)
    
def getRandomImage(path):
        #laoding random sample
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path,x)),os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class -" + face_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path,f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path + "/" + image_name)
    
for i in range(0,10):
    input_im = getRandomImage("D:\\New folder\\veer\\test\\")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original,None,fx=0.5,fy=0.5,interpolation=cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im,(224,224),interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255
    input_im = input_im.reshape(1,224,224,3)
    
    res = np.argmax(classifier.predict(input_im,1,verbose = 0),axis=1)
    draw_test("Prediction",res,input_original)
    cv2.waitKey(0)
    
cv2.destroyAllWindows()    